# Лабораторная работа №5

Провести классификацию найденного датасета, методами решающего дерева и случайного леса . В формате Markdown написать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

## метод решающего дерева

Гиперпараметры:

- max_depth: Максимальная глубина дерева. .
- min_samples_split: Минимальное количество выборок, необходимых для разделения внутреннего узла.
- min_samples_leaf: Минимальное количество выборок, необходимых для существования листового узла.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tqdm import tqdm

# Загрузка датасета
file_path = "Spotify Tracks Genre.csv"
df = pd.read_csv(file_path)
df.dropna(inplace=True)

# Разделение на признаки (X) и целевую переменную (y)
X = df[['popularity', 'duration_ms']]
y = df['mode']

# Разделение данных на тренировочный и тестовый наборы
# random_state=42 - гарантирует, что данные каждый раз будут одинакого разбиваться
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Масштабирование признаков (нормализация)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

param_grid_tree = {
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
# Создание модели решающего дерева
decision_tree = DecisionTreeClassifier()

# Поиск лучших параметров для решающего дерева
grid_search_tree = GridSearchCV(decision_tree, param_grid_tree, refit=True, verbose=0, cv=5)
grid_search_tree.fit(X_train_scaled, y_train)

# Лучшие параметры для решающего дерева
best_params_tree = grid_search_tree.best_params_
best_score_tree = grid_search_tree.best_score_

best_tree = DecisionTreeClassifier(**best_params_tree)
best_tree.fit(X_train_scaled, y_train)
accuracy_tree = best_tree.score(X_test_scaled, y_test)
print("Лучшие параметры для решающего дерева:", best_params_tree)
print("Точность решающего дерева на тестовом наборе:", accuracy_tree)

predictions = best_tree.predict(X_test_scaled)

# Отчет по классификации
print("\nОтчет по классификации:")
print(classification_report(y_test, predictions))

Лучшие параметры для решающего дерева: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2}
Точность решающего дерева на тестовом наборе: 0.6919736842105263

Отчет по классификации:
              precision    recall  f1-score   support

           0       0.58      0.58      0.58      8282
           1       0.76      0.76      0.76     14518

    accuracy                           0.69     22800
   macro avg       0.67      0.67      0.67     22800
weighted avg       0.69      0.69      0.69     22800



## метод случаного леса

Гиперпараметры:

- n_estimators: Количество деревьев в лесу. Большее количество деревьев может улучшить производительность, но с большими затратами на вычислительные ресурсы.
- max_depth: Максимальная глубина каждого дерева в лесу. Это ограничивает глубину каждого дерева в лесу, что помогает управлять переобучением.
- min_samples_split: Минимальное количество выборок, необходимых для разделения внутреннего узла дерева. Определяет, сколько выборок должно быть в узле, чтобы он был разделен.
- min_samples_leaf: Минимальное количество выборок, необходимых для существования листового узла. Это определяет, сколько выборок должно быть на каждом листе дерева.

In [2]:
# Определение параметров для случайного леса
# Указываем маленькие значения для ускорения обучения
param_grid_forest = {
    'n_estimators': range(2, 10),
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2, 4]
}
# Создание модели случайного леса
random_forest = RandomForestClassifier()

# Поиск лучших параметров для случайного леса
grid_search_forest = GridSearchCV(random_forest, param_grid_forest, refit=True, verbose=0, cv=5)
grid_search_forest.fit(X_train_scaled, y_train)

# Лучшие параметры для случайного леса
best_params_forest = grid_search_forest.best_params_
best_score_forest = grid_search_forest.best_score_
best_forest = RandomForestClassifier(**best_params_forest)
best_forest.fit(X_train_scaled, y_train)
accuracy_forest = best_forest.score(X_test_scaled, y_test)
print("Лучшие параметры для случайного леса:", best_params_forest)
print("Точность случайного леса на тестовом наборе:", accuracy_forest)

predictions = best_forest.predict(X_test_scaled)

# Отчет по классификации
print("\nОтчет по классификации:")
print(classification_report(y_test, predictions))

Лучшие параметры для случайного леса: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 9}
Точность случайного леса на тестовом наборе: 0.6826315789473684

Отчет по классификации:
              precision    recall  f1-score   support

           0       0.57      0.55      0.56      8282
           1       0.75      0.76      0.75     14518

    accuracy                           0.68     22800
   macro avg       0.66      0.65      0.65     22800
weighted avg       0.68      0.68      0.68     22800

